# Infer ***cytosol*** - part 3️⃣
(🚨🚨🚨🚨 Steps 4-9 depend on establishing a good solution here.)

--------------

## OBJECTIVE:  Infer sub-cellular component #3: ***cytosol***  in order to understand interactome 

To measure shape, position, and size of the cytosol of the cell body -- the cytosol minus the nucleus.    

Dependencies:
The ***cytoso*** inference rely on the ***nuclei*** AND ***cytosol*** inference.  Therefore all of the sub-cellular objects rely on this segmentation.





# IMPORTS

In [6]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np

from skimage.morphology import binary_erosion, binary_dilation
# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_czi_image,
                                                                    read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc_2d.utils.img import *

from infer_subc_2d.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROXI_CH ,
                                                                    ER_CH ,
                                                                    LIPID_CH ,
                                                                    RESIDUAL_CH )          

from infer_subc_2d.organelles.nuclei import infer_nuclei
from infer_subc_2d.organelles.cytosol import infer_soma

%load_ext autoreload
%autoreload 2

test_img_n = 5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [7]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [8]:

img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


In [9]:
# make sure we have removed Z
if len(scale)>2:
    scale = scale[1:]

## Now get the single "optimal" slice of all our organelle channels and infer the SOMA

takes ~ 4 seconds to calculate

In [12]:
ch_to_agg = ( LYSO_CH ,
                        MITO_CH ,
                        GOLGI_CH ,
                        PEROXI_CH ,
                        ER_CH ,
                        LIPID_CH )
                            
nuc_ch = NUC_CH
optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 


In [13]:

img_2D = img_data[:,[optimal_Z],:,:].copy()

soma_mask =  infer_soma(img_2D) 

## get the inferred nuclei object

(takes < 1 sec)

In [14]:
nuclei_object =  infer_nuclei(img_2D, soma_mask) 


# IMAGE PROCESSING Objective 3:  infer ***cytosol***

## summary of steps

➡️ INPUT
- labeled ***nuclei*** (objective #1)
- labeled ***soma*** (objective #2)

PRE-PROCESSING
- erode ***nuclei*** (shrink)

CORE PROCESSING
  - XOR(***nuclei***, ***soma***)

- POST-PROCESSING
  - N/A

OUTPUT ➡️ 
- mask of ***cytosol***
- ***cytosol*** object (intensity)


## PRE-PROCESSING

In [15]:
###################
# PRE_PROCESSING
###################

#nuclei_eroded1 = morphology.binary_erosion(nuclei_object,  footprint=morphology.ball(3) )
nuclei_eroded = binary_erosion(nuclei_object)  



## CORE PROCESSING

In [16]:

###################
# CORE_PROCESSING
###################


cyto_object = np.logical_and(soma_mask,~nuclei_eroded)

cyto_object_xor = np.logical_xor(soma_mask,nuclei_eroded)

#cyto_object = binary_dilation(cyto_object)

In [17]:



viewer = napari.view_image( #viewer.add_image(
    cyto_object,
    scale=scale,
    opacity=0.3,
)    

viewer.add_image(
    nuclei_eroded,
    scale=scale,
    opacity=0.3,
)    

viewer.add_image(
    nuclei_object,
    scale=scale,
    opacity=0.3,
)    


<Image layer 'NU_object' at 0x151c71340>

# DEFINE `_infer_cytosol` function

Based on the _prototyping_ above define the function to infer cytosol.  


In [16]:
# copy this to base.py for easy import

def _infer_cytosol(nuclei_object, soma_mask, erode_nuclei = True):
    """
    Procedure to infer cytosol from linearly unmixed input.

    Parameters:
    ------------
    nuclei_object: np.ndarray
        a 3d image containing the nuclei signal

    soma_mask: np.ndarray
        a 3d image containing the soma signal

    erode_nuclei: bool
        should we erode?

    Returns:
    -------------
    cytosol_mask: np.ndarray (bool)
      
    """

    if erode_nuclei:
        # cytosol_mask = np.logical_and(soma_mask, ~binary_erosion(nuclei_object))
        cytosol_mask = np.logical_xor(soma_mask, binary_erosion(nuclei_object))
    else:
        # cytosol_mask = np.logical_and(soma_mask, ~nuclei_object)
        cytosol_mask = np.logical_xor(soma_mask, nuclei_object)


    return cytosol_mask


--------------------------

# TEST `infer_cytosol` exported functions


##
`infer_cytosol` procedure

In [18]:
from infer_subc_2d.organelles.cytosol import infer_cytosol

_cytosol_mask =  _infer_cytosol(soma_mask, nuclei_object) 

cytosol_mask =  infer_cytosol(soma_mask, nuclei_object) 


In [22]:


viewer.scale_bar.visible = True
viewer.add_image(
    cytosol_mask,
    scale=scale 
)

viewer.add_labels(
    _cytosol_mask,
    scale=scale 
)

viewer.add_labels(
    nuclei_object,
    scale=scale 
)

viewer.add_image(
    nuclei_eroded,
    scale=scale 
)

<Image layer 'NU_eroded' at 0x183afcfd0>

In [ ]:
from napari.utils.notebook_display import nbscreenshot

# viewer.dims.ndisplay = 3
# viewer.camera.angles = (-30, 25, 120)
nbscreenshot(viewer, canvas_only=True)

--------------------------
## Write workflow .json
Now that we've added our function specs we can compose workflows.

In [ ]:
def make_infer_cytosol_dict():
    """
    Procedure to infer infer from linearly unmixed input. (logical soma AND NOT nucleus)

    Parameters:
    ------------
    nuclei_object: np.ndarray
        a 3d image containing the nuclei object

    soma_mask: np.ndarray
        a 3d image containing the soma object (mask)

    erode_nuclei: bool
        should we erode?

    Returns:
    -------------
    cytosol_mask: np.ndarray (bool)

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    
    # eroded_nuclear_object = binary_erosion(nuclei_object)


    step_name.append("1")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(0)

    # cytosol_mask = np.logical_xor(soma_mask,eroded_nuclear_object)

    step_name.append("2")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(1)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_cytosol_dict = make_infer_cytosol_dict()

write_workflow_json("infer_cytosol", infer_cytosol_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/infer_nuclei.json')

## make function json to add to `all_functions.json`
### infer_nuclei 

In [ ]:
from infer_subc_2d.organelles_config.helper import add_function_spec_to_widget_json

_infer_soma =  {
        "name": "infer infer_soma",
        "python::module": "infer_subc_2d.organelles",
        "python::function": "infer_soma",
        "parameters": None
        }

add_function_spec_to_widget_json("infer_soma",_infer_soma)

1